[View in Colaboratory](https://colab.research.google.com/github/timohe/ML-Keras_ConvNeuralNet/blob/master/cifar10_cnn_exam_solution.ipynb)

# CIFAR10 with CNN
code from https://github.com/fchollet/keras/blob/master/examples/mnist_cnn.py


This notebook trains a simple convolutional neural network on the CIFAR10 small images dataset. 



In [ ]:
# import libraries
from __future__ import print_function
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D

#for confusion matrix
import numpy as np
import sklearn
from sklearn.metrics import confusion_matrix


Prepare data

In [ ]:
# define constants
batch_size = 128
num_classes = 10
epochs = 1


# The data, shuffled and split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# Cast features into correct data type then scale features
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

Define model

In [ ]:
model = Sequential()

model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))


Show model structure

In [ ]:
model.summary()

Compile model and fit


In [ ]:
# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
                metrics=['accuracy'])


model.fit(x_train, y_train,
            batch_size=batch_size,
            epochs=epochs,
            validation_data=(x_test, y_test),
            shuffle=True)




In [ ]:
# Save the original model
model.save('savedFiles/cifra10_base.h5')

In [ ]:
# Print base model loss and accuracy
print('\n  - Base case:')
model = keras.models.load_model("savedFiles/cifra10_base.h5")
scores = model.evaluate(x_test, y_test, verbose=0)
print('\tTest loss:\t', scores[0])
print('\tTest accuracy:\t', scores[1])
Y_pred = model.predict(x_test, verbose=2)
cm = confusion_matrix(np.argmax(y_test,axis=1),np.argmax(Y_pred, axis=1))
print('\tConfusion Matrix:\t')
print(cm)

### 1) Augment the data by adding noise. Discuss results.

In [ ]:
# Save models with gaussian noise with different Standard Deviation
for e in [0.08,0.07,0.06,0.05,0.04,0.03,0.02,0.01]:
    model = keras.models.load_model("savedFiles/cifra10_base.h5")
    model.add(keras.layers.GaussianNoise(e))
    model.compile(loss='categorical_crossentropy',
                  optimizer=opt,
                    metrics=['accuracy'])
    model.fit(x_train, y_train,
                batch_size=batch_size,
                epochs=epochs,
                validation_data=(x_test, y_test),
                shuffle=True)
    model.save('savedFiles/cifra10_n'+str(e)+'.h5')

In [ ]:
# Load models and print results
for e in [0.08,0.07,0.06,0.05,0.04,0.03,0.02,0.01]:
    model = keras.models.load_model('savedFiles/cifra10_n'+str(e)+'.h5')
    print('\n  - Gaussian Noise with '+str(e)+' stdev:')
    scores = model.evaluate(x_test, y_test, verbose=0)
    print('\tTest loss:\t', scores[0])
    print('\tTest accuracy:\t', scores[1])
    
# Load and print of the best loss and accuracy obtained
model = keras.models.load_model("savedFiles/cifra10_n0.02.h5")
print('\n\nBest results:')
print('  - Gaussian Noise with 0.02 stdev:')
scores = model.evaluate(x_test, y_test, verbose=0)
print('\tTest loss:\t', scores[0])
print('\tTest accuracy:\t', scores[1])

### 2) Add layers to the network to see if can improve performance

#### First model: batch normalization

In [ ]:
from keras.layers.normalization import BatchNormalization

model = Sequential()

model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))

model.add(BatchNormalization())

model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))

model.add(BatchNormalization())

model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))

model.add(BatchNormalization())

model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))

model.add(BatchNormalization())

model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer=opt,
                metrics=['accuracy'])
model.fit(x_train, y_train,
            batch_size=batch_size,
            epochs=epochs,
            validation_data=(x_test, y_test),
            shuffle=True)

model.save('savedFiles/cifra10_batch.h5')


In [ ]:

model = keras.models.load_model('savedFiles/cifra10_batch.h5')
print('\n  - Model with Batch Normalization:')
scores = model.evaluate(x_test, y_test, verbose=0)
print('\tTest loss:\t', scores[0])
print('\tTest accuracy:\t', scores[1])

Y_pred = model.predict(x_test, verbose=2)
cm = confusion_matrix(np.argmax(y_test,axis=1),np.argmax(Y_pred, axis=1))
print('\tConfusion Matrix:\t')
print(cm)

#### Second model: regularizer

In [ ]:
from keras.layers.normalization import BatchNormalization
from keras import regularizers

weight_decay = 1e-4

model = Sequential()

model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))

model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer=opt,
                metrics=['accuracy'])
model.fit(x_train, y_train,
            batch_size=batch_size,
            epochs=epochs,
            validation_data=(x_test, y_test),
            shuffle=True)

model.save('savedFiles/cifra10_regL2.h5')

In [ ]:
model= keras.models.load_model('savedFiles/cifra10_regL2.h5')
print('\n  - regularizer L2:')
scores = model.evaluate(x_test, y_test, verbose=0)
print('\tTest loss:\t', scores[0])
print('\tTest accuracy:\t', scores[1])

Y_pred = model.predict(x_test, verbose=2)
cm = confusion_matrix(np.argmax(y_test,axis=1),np.argmax(Y_pred, axis=1))
print('\tConfusion Matrix:\t')
print(cm)

### 3) Try another method for dealing with overfitting

In [ ]:
from keras.layers.normalization import BatchNormalization
from keras import regularizers

weight_decay = 1e-4

model = Sequential()

model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))

model.add(BatchNormalization())

model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))

model.add(BatchNormalization())

model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))

model.add(BatchNormalization())

model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(weight_decay)))

model.add(BatchNormalization())

model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer=opt,
                metrics=['accuracy'])
model.fit(x_train, y_train,
            batch_size=batch_size,
            epochs=epochs,
            validation_data=(x_test, y_test),
            shuffle=True)

model.save('savedFiles/cifra10_batch_regL2.h5')

In [ ]:
model.save('savedFiles/cifra10_batch_regL2.h5')
print('\n  - Model with batch normalization and regularizer:')
scores = model.evaluate(x_test, y_test, verbose=0)
print('\tTest loss:\t', scores[0])
print('\tTest accuracy:\t', scores[1])

In [ ]:
model = keras.models.load_model('savedFiles/cifra10_batch.h5')
model.add(keras.layers.GaussianNoise(0.01))
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
                metrics=['accuracy'])
model.fit(x_train, y_train,
            batch_size=batch_size,
            epochs=epochs,
            validation_data=(x_test, y_test),
            shuffle=True)
model.save('savedFiles/cifra10_batch_n0.01.h5')

In [ ]:
model = keras.models.load_model('savedFiles/cifra10_batch_n0.01.h5')

print('\n  - Model with Batch Normalization and Gaussian Noise with 0.01 stdev:')
scores = model.evaluate(x_test, y_test, verbose=0)
print('\tTest loss:\t', scores[0])
print('\tTest accuracy:\t', scores[1])